In [ ]:
import pandas as pd
import pickle
from pathlib import Path
from collections import Counter

In [ ]:
g2p_path = 'path_to_g2p_entries.csv'
g2p_df = pd.read_csv(g2p_path)
g2p_df = g2p_df.drop(columns=['phenotypes', 'organ specificity list', 'panel',
       'prev symbols', 'gene disease pair entry date'])

ret_df = pickle.load(open('ret_mm_df.p','rb'))

In [ ]:
def model_per_g2p_entry(g2p_df, retrieval_df_mm_processed, retrieval_df_model_series_name):
    g2p_model_list = []
    for pmid_str in g2p_df['pmids']:
        try:
            pmid_list = pmid_str.split(';')
            g2p_from_retrieval_df = retrieval_df_mm_processed[retrieval_df_mm_processed['pmid'].isin(pmid_list)]
            all_freq = [i for j in g2p_from_retrieval_df[retrieval_df_model_series_name] for i in j]
            exp_freq = [i for hp, n in all_freq for i in [hp] * n]
            total_hp = Counter(exp_freq).most_common()

            g2p_model_list.append(total_hp)
        
        except:
            g2p_model_list.append(None)
    
    return g2p_model_list

In [ ]:
g2p_df['tiab_hpo'] = model_per_g2p_entry(g2p_df,ret_df,'tiab_hpo')
g2p_df['content_hpo'] = model_per_g2p_entry(g2p_df,ret_df,'content_hpo')

In [ ]:
pickle.dump(g2p_df,open('g2p_df_mm.p','wb'))